In [ ]:
import pdfplumber
import requests
import numpy as np
import pandas as pd


In [ ]:
url = "https://evs.nci.nih.gov/ftp1/CTCAE/CTCAE_4.03/Archive/CTCAE_4.0_2009-05-29_QuickReference_8.5x11.pdf"
filename = "CTCAE_v4.pdf"
response = requests.get(url)
with open(filename, 'wb') as f:
    f.write(response.content)

In [ ]:
ctcae = pd.DataFrame(columns=["Name", "Grade1", "Grade2", "Grade3", "Grade4", "Grade5", "Definition", "Group"])

pdf_path = "CTCAE_v4.pdf" # make sure this is the correct path to the PDF file
with pdfplumber.open(pdf_path) as pdf:
    print(f"PDF has {len(pdf.pages)} pages")
    
    # Check first few pages for tables
    for i in np.arange(2, 78): # we only give here the pages that include tables
        print(i)
        page = pdf.pages[i]
        title = page.extract_tables()[1][0][0] # grand title of the page
        for row in page.extract_tables()[1][3:]:
            row = [i.replace("\n", " ") for i in row if i is not None]
            if "Definition:" not in row[0]:
                new_row = {
                    "Name": row[0],
                    "Grade1": row[1],
                    "Grade2": row[2],
                    "Grade3": row[3],
                    "Grade4": row[4],
                    "Grade5": row[5],
                }
                ctcae = pd.concat([ctcae, pd.DataFrame([new_row])])
                ctcae.reset_index(drop=True, inplace=True)
            else:
                ctcae.loc[ctcae.index[-1],'Definition'] = row[0].split("Definition: ")[1] # the last entry of Definition
            ctcae.loc[ctcae.index[-1],'Group'] = title


In [ ]:
ctcae.to_excel("CTCAE_v4.xlsx", index=False)